# QAS Demo

This notebook is adapted for use in the Gradient Climber application using the Garden Buddy application as an example.  We use the Gradient Climber Quality Attribute Scenarios as guidence for the required Properties and Conditions.

NOTE: this demo has an additional set of requirements than MLTE. You can install them with the command: 

`poetry install --with demo`


## 0. Quality Attribute Scenarios

The following are the QASs that we want to validate through the use of MLTE. The examples below relate to a reinforcement learning system based on the mountain car example [Andrew W. Moore (1990). Efficient Memory-Based Learning for Robot Control (PhD thesis, University of Cambridge)] ().

* **Interpretability - Model Produces Valid Outputs**
The model always produces outputs according to the output actions specified in the  documentation during normal operation


* **Functional Correctness - Model Does the Expected Thing**
Agent receives "start" command from system initialized with random position during Normal Operation. Agent moves car to desired location of x=0.6 within 250 time steps.
After initialization, the model successfully navigates the vehicle from a stationary position at the bottom of a hill, to the desired location (top of the hill in front of the vehicle) within the specified time bound.

* **Deployability - Model Can be Deployed into Productin When Needed**
New version of agent Q-table deployed onto device during Maintenance. New q-table is loaded into into available memory and system produces outputs within 16ms (60hz).

* **Reliability - Model Performs as Exected without Failure**
Model receives valid values from sensors during Normal Operation and produces outputs (actions) which improve the expected reward 99.9% of the time.
Model receives valid input values from from sensors during normal operation with the gradient climber system active. The model output is correct and improves the expected reward 99.9% of the time.

* **Portability - Model can be Adatped for Use on New Hardware/Software Platforms**
A new model is generated during the model training process. The model should be serialized in a format that can be moved across platforms with no need for retraining 

* **Compliance (Speed) - Model Adheres to Designated Ethical and Saftey Requirements**
The model receives a position and velocity input from *source* during normal operations. Regardless of input, the model will not produce output values that cause the vehicle to exceed the legal speed limit set in the operational configuration.

* **Compliance (Position) - Model Adheres to Designated Ethical and Saftey Requirements**
The model receives a position and velocity from *source* during normal operations. Regardless of input, the model will not produce output values that cause the vehicle to exit the area of operation set in the operational configuration.

# 1. Requirements

## 1.1 Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [1]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from demo.GradientClimber.session import *

Creating initial custom lists at URI: local:///Users/jhansen/continuum/mlte/demo/GradientClimber/../store
Loaded 7 qa_categories for initial list
Loaded 30 quality_attributes for initial list
Creating sample catalog at URI: StoreType.LOCAL_FILESYSTEM:local:///Users/jhansen/continuum/mlte/demo/GradientClimber/../store
Loading sample catalog entries.
Loaded 9 entries for sample catalog.


## 1.2 Build a `NegotiationCard`

In MLTE, we negotiation requirements with the help of a `NegotiationCard`. This can be done manually through code, but it is easier to use the MLTE UI to do so. Below we are copying a pre-built one that applies to this scenario. In MLTE, we define requirements by constructing a `NegotiationCard` that will include explicit Quality Attribute Scenarios with the requirements.

In [2]:
# JPH - Is this needed for mountain car?
#%%bash
#(cd ../ && bash setup_store.sh)

## 1.3 Define a TestSuite

In the first phase of SDMT, we define a `TestSuite` that represents the tests the completed model must will have to pass in order to be acceptable for use in the system into which it will be integrated.

In MLTE, we define the tests that will be required for the different requirements in a `TestSuite`. Note that a new `Evidence` types (`MultipleRanksums`) had to be created in this case to handle the data and `Validator` for that case, and two stand-alone `Validator`s were defined in `validators.py` to validate data using existing `Evidence` types.

First we need to load up our `NegotiationCard`, so we can get the list of ids of its quaity attribute scenarios, that will be added to the `TestCase`s here. Those ids are the way to link the `TestCase`s to their quality attribute requirements.

In [3]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
card.save(force=True)
card.print_quality_scenarios()

ErrorNotFound: Artifact card.default

Now we can create our `TestSuite`, consisting of a list of `TestCases`, each of them addressing one or more Quality Attribute Scenarios from our `NegotiationCard`. When defining the `TestCase`s below, we need to set the id of the corresponding Quality Attribute Scenario we want to test in its "quality_scenarios" attribute.

In [ ]:
from mlte.tests.test_case import TestCase
from mlte.tests.test_suite import TestSuite

# The Evidence types we will use to validate each condition.
from mlte.measurement.storage import LocalObjectSize
from mlte.measurement.cpu import LocalProcessCPUUtilization
from mlte.measurement.units import Units
from mlte.measurement.memory import LocalProcessMemoryConsumption
from mlte.evidence.types.image import Image
from mlte.evidence.types.string import String
from mlte.evidence.types.real import Real
from demo.scenarios import validators
from demo.scenarios.evidence.multiple_ranksums import MultipleRanksums
from mlte.evidence.types.string import String
from mlte.validation.validator import Validator

test_suite = TestSuite(
    test_cases=[
        # Complience (position)
        TestCase(
            identifier="gradient climber position accuracy",
            goal="Check if model complies with position requirements",
            quality_scenarios=["default.card-qas_003"],
            validator=validators.all_accuracies_more_or_equal_than(0.9)
        )
    ]
)
# The full test suite.
"""
#
# Garden buddy example for reference
#
test_suite = TestSuite(
    test_cases=[
        # Fairness QAS test cases.
        TestCase(
            identifier="accuracy across gardens",
            goal="Check if model performs well accross different populations",
            quality_scenarios=["default.card-qas_001"],
            validator=validators.all_accuracies_more_or_equal_than(0.9),
        ),
        # Robustness QAS test cases.
        TestCase(
            identifier="ranksums blur2x8",
            goal="Check blur and noise for 2x8 case",
            quality_scenarios=["default.card-qas_002"],
            validator=validators.p_value_greater_or_equal_to(0.05 / 3),
        ),
        TestCase(
            identifier="ranksums blur5x8",
            goal="Check blur and noise for 5x8 case",
            quality_scenarios=["default.card-qas_002"],
            validator=validators.p_value_greater_or_equal_to(0.05 / 3),
        ),
        TestCase(
            identifier="ranksums blur0x8",
            goal="Check blur and noise for 0x8 case",
            quality_scenarios=["default.card-qas_002"],
            validator=validators.p_value_greater_or_equal_to(0.05 / 3),
        ),
        TestCase(
            identifier="effect of blur across families",
            goal="Check consistency in families",
            quality_scenarios=["default.card-qas_002"],
            validator=MultipleRanksums.all_p_values_greater_or_equal_than(
                0.05 / 141
            ),
        ),
        # Resilience QAS test case
        TestCase(
            identifier="ranksums channel loss R",
            goal="Check consistency between channel loss",
            quality_scenarios=["default.card-qas_003"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        TestCase(
            identifier="ranksums channel loss G",
            goal="Check consistency between channel loss",
            quality_scenarios=["default.card-qas_003"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        TestCase(
            identifier="ranksums channel loss B",
            goal="Check consistency between channel loss",
            quality_scenarios=["default.card-qas_003"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        # Resource Utilization QASs test cases.
        TestCase(
            identifier="model size",
            goal="Check storage consumption",
            quality_scenarios=["default.card-qas_004"],
            validator=LocalObjectSize.get_output_type().less_than(
                150, Units.megabyte
            ),
        ),
        TestCase(
            identifier="predicting memory",
            goal="Check memory used while predicting",
            quality_scenarios=["default.card-qas_004"],
            validator=LocalProcessMemoryConsumption.get_output_type().average_consumption_less_than(
                512.0, unit=Units.megabyte
            ),
        ),
        TestCase(
            identifier="predicting cpu",
            goal="Check cpu % used while predicting",
            quality_scenarios=["default.card-qas_004"],
            validator=LocalProcessCPUUtilization.get_output_type().max_utilization_less_than(
                30.0, unit=Units.percent
            ),
        ),
        # Understandability QAS test case.
        TestCase(
            identifier="image attributions",
            goal="Check what the model is doing",
            quality_scenarios=["default.card-qas_005"],
            validator=Image.register_info("Inspect the image."),
        ),
        # Functional Correctness - Accuracy QAS test cases.
        TestCase(
            identifier="overall model accuracy",
            goal="Measure the overall accuracy of your end to end pipeline",
            quality_scenarios=["default.card-qas_006"],
            validator=Real.greater_than(0.9),
        ),
        # Functional Correctness - I/O spec QAS test cases.
        TestCase(
            identifier="input format validation success",
            goal="Model input format must conform to specified format",
            quality_scenarios=["default.card-qas_007"],
            validator=String.contains("Model - Input Validation Pass"),
        ),
        TestCase(
            identifier="output format validation success",
            goal="Model output format must conform to specified format",
            quality_scenarios=["default.card-qas_007"],
            validator=String.contains("Model - Output Validation Pass"),
        ),
        # Reliability: Input Validation QAS test cases.
        TestCase(
            identifier="input format validation error",
            goal="Model inputs must conform to specified format",
            quality_scenarios=["default.card-qas_008"],
            validator=String.contains("Model - Input Validation Error"),
        ),
        #  Analyzability QAS test cases.
        TestCase(
            identifier="detect ood inputs",
            goal="Monitor inputs for OOD data and unexpected shifts",
            quality_scenarios=["default.card-qas_009"],
            validator=String.contains("Model - Input OOD Error"),
        ),
        # Monitorability QAS test case
        TestCase(
            identifier="monitor output confidence shift",
            goal="Monitor inputs for OOD data and unexpected shifts",
            quality_scenarios=["default.card-qas_010"],
            validator=String.contains("Model - Output Confidence Error"),
        ),
        # Time Behaviour
        TestCase(
            identifier="predicting cpu time",
            goal="Check cpu time used while predicting",
            quality_scenarios=["default.card-qas_011"],
            validator=Real.less_than(2.0, Units.second),
        ),
        # Repeatability QAS test case
        TestCase(
            identifier="repeated results sampling",
            goal="Repeatedly sampling results gives same results",
            quality_scenarios=["default.card-qas_012"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        # Reproducibility QAS test case
        TestCase(
            identifier="repeated training on training samples",
            goal="Repeatedly training on different sammples of training data gives same results on test data set",
            quality_scenarios=["default.card-qas_013"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        # Domain Adaptability QAS test case
        TestCase(
            identifier="running in new domain",
            goal="Repeatedly training on different sammples of training data gives same results on test data set",
            quality_scenarios=["default.card-qas_014"],
            validator=MultipleRanksums.all_p_values_greater_or_equal_than(0.05),
        ),
        # Testability QAS test case
        TestCase(
            identifier="test results from dev and op env",  # test results from dev and op env
            goal="aligment of test results from dev and op environments",
            quality_scenarios=["default.card-qas_015"],
            validator=Validator.build_info_validator(
                "Inspect the alignment of results for no more than 0.25% difference."
            ),
        ),
        # Understandability QAS test case
        TestCase(
            identifier="understanding design choices",
            goal="understanding design and implementation choices",
            quality_scenarios=["default.card-qas_016"],
            validator=Validator.build_info_validator(
                "Inspect project code and documentation."
            ),
        ),
        # Maintainability QAS test case
        TestCase(
            identifier="keep ML component up to date",
            goal="keep trained ML component up to date with op environment changes",
            quality_scenarios=["default.card-qas_017"],
            validator=Validator.build_info_validator(
                "Validate work time less than 8hrs."
            ),
        ),
        # Modifiability QAS test case
        TestCase(
            identifier="update data pipelines",
            goal="aligment of test results from dev and op environments",
            quality_scenarios=["default.card-qas_018"],
            validator=Validator.build_info_validator(
                "Validate work time less than 4hrs."
            ),
        ),
        # Replaceability QAS test case
        TestCase(
            identifier="update ML training algorithm",
            goal="update ML component training algorithm",
            quality_scenarios=["default.card-qas_019"],
            validator=Validator.build_info_validator(
                "Validate train time less than 16hrs."
            ),
        ),
        # Retrainability QAS test case
        TestCase(
            identifier="retrain ML model",
            goal="update ML component through training",
            quality_scenarios=["default.card-qas_020"],
            validator=Validator.build_info_validator(
                "Model takes less than 1 hr to retrain."
            ),
        ),
        # Reuseability QAS test case
        TestCase(
            identifier="reuse ML component",
            goal="reuse ML component in new app",
            quality_scenarios=["default.card-qas_021"],
            validator=Validator.build_info_validator(
                "Validate work time less than 4hrs."
            ),
        ),
    ]
)
"""
test_suite.save(parents=True, force=True)

ArtifactModel(header=ArtifactHeaderModel(identifier='suite.default', type='suite', timestamp=1760109019, creator=None, level='model'), body=TestSuiteModel(artifact_type=<ArtifactType.TEST_SUITE: 'suite'>, test_cases=[TestCaseModel(identifier='accuracy across gardens', goal='Check if model performs well accross different populations', qas_list=['default.card-qas_001'], measurement=None, validator=ValidatorModel(bool_exp='gASVtwIAAAAAAACMCmRpbGwuX2RpbGyUjBBfY3JlYXRlX2Z1bmN0aW9ulJOUKGgAjAxfY3JlYXRlX2NvZGWUk5QoQwgMAQT/BgII/pRLAUsASwBLAUsESxNDInQAhwBmAWQBZAKECHwAagFEAIMBgwF0AnwAagGDAWsCUwCUTmgEKEMEBgEI/5RLAUsASwBLAksDSzNDGIEAfABdB30BfAGIAGsFVgABAHECZABTAJROhZQpjAIuMJSMAWeUhpSMOi9Vc2Vycy9qaGFuc2VuL2NvbnRpbnV1bS9tbHRlL2RlbW8vc2NlbmFyaW9zL3ZhbGlkYXRvcnMucHmUjAk8Z2VuZXhwcj6USxJDCAKABAAIAQr/lIwJdGhyZXNob2xklIWUKXSUUpSMRmFsbF9hY2N1cmFjaWVzX21vcmVfb3JfZXF1YWxfdGhhbi48bG9jYWxzPi48bGFtYmRhPi48bG9jYWxzPi48Z2VuZXhwcj6Uh5SMA3N1bZSMBWFycmF5lIwDbGVulIeUjAV2YWx1ZZSFlGgNjAg8bGFtYmRhPpRLEkMKDAAEAQb/CAIE/pRo